In [1]:
import os
import json
from PIL import Image

In [ ]:
# training 데이터
def convert_to_ocr_format(bboxes, image_width, image_height):
    """
    절대 좌표를 OCR 포맷으로 변환.
    Args:
        bboxes: 리스트 형태의 절대 좌표 [(class_id, [x_min, y_min, x_max, y_max]), ...]
        image_width: 이미지 너비
        image_height: 이미지 높이
    Returns:
        ocr_bboxes: OCR 포맷으로 변환된 리스트 [(class_id, x_center, y_center, width, height), ...]
    """
    ocr_bboxes = []
    for bbox in bboxes:
        class_id, coords = bbox[0], bbox[1]
        x_min, y_min, x_max, y_max = coords
        x_center = (x_min + x_max) / 2 / image_width
        y_center = (y_min + y_max) / 2 / image_height
        width = (x_max - x_min) / image_width
        height = (y_max - y_min) / image_height
        ocr_bboxes.append((class_id, x_center, y_center, width, height))
    return ocr_bboxes

def process_json_to_txt_with_image(input_folder, image_folder, output_folder):
    """
    JSON 파일을 TXT 파일로 변환하고 폴더 구조를 유지하여 저장, 이미지 크기를 매핑하여 계산.
    Args:
        input_folder (str): JSON 파일이 포함된 최상위 폴더.
        image_folder (str): 이미지 파일이 포함된 최상위 폴더.
        output_folder (str): TXT 파일을 저장할 최상위 폴더.
    """
    skipped_files = []  # 생성 실패한 파일들을 저장

    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.endswith(".json"):
                json_path = os.path.join(root, file)
                relative_path = os.path.relpath(root, input_folder)
                output_dir = os.path.join(output_folder, relative_path)
                os.makedirs(output_dir, exist_ok=True)
                
                txt_path = os.path.join(output_dir, file.replace(".json", ".txt"))
                
                # JSON 파일 읽기
                try:
                    with open(json_path, "r", encoding="utf-8-sig") as json_file:
                        data = json.load(json_file)
                except json.JSONDecodeError:
                    skipped_files.append(f"JSON 파일 손상: {json_path}")
                    continue
                
                # 비어있는 JSON 파일 확인
                if not data.get("OCR_info"):
                    skipped_files.append(f"비어있는 JSON 파일: {json_path}")
                    continue

                # 이미지 경로 설정 (JSON 파일과 동일한 이름의 이미지 검색)
                image_path = os.path.join(image_folder, relative_path, file.replace(".json", ".png"))
                if not os.path.exists(image_path):
                    skipped_files.append(f"이미지 파일 없음: {json_path}")
                    continue
                
                # 이미지 크기 읽기
                try:
                    image = Image.open(image_path)
                    image_width, image_height = image.size
                except Exception as e:
                    skipped_files.append(f"이미지 파일 읽기 실패: {image_path} ({e})")
                    continue

                # 절대 좌표 수집
                absolute_bboxes = []
                has_image = False
                for ocr in data.get("OCR_info", []):
                    for bbox in ocr.get("question_bbox", []):
                        bbox_coords = bbox.get("bbox", [])
                        
                        # "line" 처리
                        if bbox.get("type") == "line":
                            absolute_bboxes.append((0, bbox_coords))  # class_id = 0 for line
                        
                        # "image" 처리
                        elif bbox.get("type") == "image":
                            absolute_bboxes.append((1, bbox_coords))  # class_id = 1 for image
                            has_image = True  # `image` 좌표가 있는지 확인

                # 절대 좌표를 OCR 포맷으로 변환
                converted_bboxes = convert_to_ocr_format(absolute_bboxes, image_width, image_height)

                # TXT 파일 작성
                with open(txt_path, "w", encoding="utf-8") as file:
                    # bounding_boxes 출력
                    file.write("[bboxs]\n")
                    for bbox in converted_bboxes:
                        class_id, x_center, y_center, width, height = bbox
                        file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
                    
                    # question_text 출력
                    question_text = data["OCR_info"][0].get("question_text", "null")
                    file.write("\n[question_text]\n")
                    file.write(f"{question_text}\n")
                    
                    # figure_text 출력 (image 좌표가 있는 경우에만 처리)
                    if has_image:
                        figure_text = data["OCR_info"][0].get("figure_text", None)
                        if figure_text is None:  # None 값 처리
                            figure_text = "null"
                        file.write("\n[figure_text]\n")
                        file.write(f"{figure_text}\n")
                
                print(f"저장 완료: {txt_path}")

    # 스킵된 파일 출력
    if skipped_files:
        print("\n처리되지 못한 파일 목록:")
        for skipped_file in skipped_files:
            print(skipped_file)

# 최상위 폴더 설정
input_folder = "C:/Users/boar2/Desktop/final_project/github/Data/images/training/y"  # JSON 파일이 포함된 최상위 폴더
image_folder = "C:/Users/boar2/Desktop/final_project/github/Data/images/training/x"  # 이미지 파일이 포함된 최상위 폴더
output_folder = "C:/Users/boar2/Desktop/final_project/github/Data/images/training/formatted_y_2"  # TXT 파일을 저장할 최상위 폴더

In [6]:
# JSON → TXT 변환 실행
process_json_to_txt_with_image(input_folder, image_folder, output_folder)

저장 완료: C:/Users/boar2/Desktop/final_project/github/Data/images/training/formatted_y_2\elementary3\P3_1_01_21114_49495.txt
저장 완료: C:/Users/boar2/Desktop/final_project/github/Data/images/training/formatted_y_2\elementary3\P3_1_01_21114_49496.txt
저장 완료: C:/Users/boar2/Desktop/final_project/github/Data/images/training/formatted_y_2\elementary3\P3_1_01_21114_49497.txt
저장 완료: C:/Users/boar2/Desktop/final_project/github/Data/images/training/formatted_y_2\elementary3\P3_1_01_21114_49506.txt
저장 완료: C:/Users/boar2/Desktop/final_project/github/Data/images/training/formatted_y_2\elementary3\P3_1_01_21116_49499.txt
저장 완료: C:/Users/boar2/Desktop/final_project/github/Data/images/training/formatted_y_2\elementary3\P3_1_01_21116_49500.txt
저장 완료: C:/Users/boar2/Desktop/final_project/github/Data/images/training/formatted_y_2\elementary3\P3_1_01_21116_49501.txt
저장 완료: C:/Users/boar2/Desktop/final_project/github/Data/images/training/formatted_y_2\elementary3\P3_1_01_21116_49502.txt
저장 완료: C:/Users/boar2/De

In [7]:
# validation 데이터
def convert_to_ocr_format(bboxes, image_width, image_height):
    """
    절대 좌표를 OCR 포맷으로 변환.
    Args:
        bboxes: 리스트 형태의 절대 좌표 [(class_id, [x_min, y_min, x_max, y_max]), ...]
        image_width: 이미지 너비
        image_height: 이미지 높이
    Returns:
        ocr_bboxes: OCR 포맷으로 변환된 리스트 [(class_id, x_center, y_center, width, height), ...]
    """
    ocr_bboxes = []
    for bbox in bboxes:
        class_id, coords = bbox[0], bbox[1]
        x_min, y_min, x_max, y_max = coords
        x_center = (x_min + x_max) / 2 / image_width
        y_center = (y_min + y_max) / 2 / image_height
        width = (x_max - x_min) / image_width
        height = (y_max - y_min) / image_height
        ocr_bboxes.append((class_id, x_center, y_center, width, height))
    return ocr_bboxes

def process_json_to_txt_with_image(input_folder, image_folder, output_folder):
    """
    JSON 파일을 TXT 파일로 변환하고 폴더 구조를 유지하여 저장, 이미지 크기를 매핑하여 계산.
    Args:
        input_folder (str): JSON 파일이 포함된 최상위 폴더.
        image_folder (str): 이미지 파일이 포함된 최상위 폴더.
        output_folder (str): TXT 파일을 저장할 최상위 폴더.
    """
    skipped_files = []  # 생성 실패한 파일들을 저장

    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.endswith(".json"):
                json_path = os.path.join(root, file)
                relative_path = os.path.relpath(root, input_folder)
                output_dir = os.path.join(output_folder, relative_path)
                os.makedirs(output_dir, exist_ok=True)
                
                txt_path = os.path.join(output_dir, file.replace(".json", ".txt"))
                
                # JSON 파일 읽기
                try:
                    with open(json_path, "r", encoding="utf-8-sig") as json_file:
                        data = json.load(json_file)
                except json.JSONDecodeError:
                    skipped_files.append(f"JSON 파일 손상: {json_path}")
                    continue
                
                # 비어있는 JSON 파일 확인
                if not data.get("OCR_info"):
                    skipped_files.append(f"비어있는 JSON 파일: {json_path}")
                    continue

                # 이미지 경로 설정 (JSON 파일과 동일한 이름의 이미지 검색)
                image_path = os.path.join(image_folder, relative_path, file.replace(".json", ".png"))
                if not os.path.exists(image_path):
                    skipped_files.append(f"이미지 파일 없음: {json_path}")
                    continue
                
                # 이미지 크기 읽기
                try:
                    image = Image.open(image_path)
                    image_width, image_height = image.size
                except Exception as e:
                    skipped_files.append(f"이미지 파일 읽기 실패: {image_path} ({e})")
                    continue

                # 절대 좌표 수집
                absolute_bboxes = []
                has_image = False
                for ocr in data.get("OCR_info", []):
                    for bbox in ocr.get("question_bbox", []):
                        bbox_coords = bbox.get("bbox", [])
                        
                        # "line" 처리
                        if bbox.get("type") == "line":
                            absolute_bboxes.append((0, bbox_coords))  # class_id = 0 for line
                        
                        # "image" 처리
                        elif bbox.get("type") == "image":
                            absolute_bboxes.append((1, bbox_coords))  # class_id = 1 for image
                            has_image = True  # `image` 좌표가 있는지 확인

                # 절대 좌표를 OCR 포맷으로 변환
                converted_bboxes = convert_to_ocr_format(absolute_bboxes, image_width, image_height)

                # TXT 파일 작성
                with open(txt_path, "w", encoding="utf-8") as file:
                    # bounding_boxes 출력
                    file.write("[bboxs]\n")
                    for bbox in converted_bboxes:
                        class_id, x_center, y_center, width, height = bbox
                        file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
                    
                    # question_text 출력
                    question_text = data["OCR_info"][0].get("question_text", "null")
                    file.write("\n[question_text]\n")
                    file.write(f"{question_text}\n")
                    
                    # figure_text 출력 (image 좌표가 있는 경우에만 처리)
                    if has_image:
                        figure_text = data["OCR_info"][0].get("figure_text", None)
                        if figure_text is None:  # None 값 처리
                            figure_text = "null"
                        file.write("\n[figure_text]\n")
                        file.write(f"{figure_text}\n")
                
                print(f"저장 완료: {txt_path}")

    # 스킵된 파일 출력
    if skipped_files:
        print("\n처리되지 못한 파일 목록:")
        for skipped_file in skipped_files:
            print(skipped_file)

# 최상위 폴더 설정
input_folder = "C:/Users/boar2/Desktop/final_project/github/Data/images/validation/y"  # JSON 파일이 포함된 최상위 폴더
image_folder = "C:/Users/boar2/Desktop/final_project/github/Data/images/validation/x"  # 이미지 파일이 포함된 최상위 폴더
output_folder = "C:/Users/boar2/Desktop/final_project/github/Data/images/validation/formatted_y_2"  # TXT 파일을 저장할 최상위 폴더

In [8]:
# JSON → TXT 변환 실행
process_json_to_txt_with_image(input_folder, image_folder, output_folder)

저장 완료: C:/Users/boar2/Desktop/final_project/github/Data/images/validation/formatted_y_2\elementary3\P3_1_01_21114_49498.txt
저장 완료: C:/Users/boar2/Desktop/final_project/github/Data/images/validation/formatted_y_2\elementary3\P3_1_01_21169_50033.txt
저장 완료: C:/Users/boar2/Desktop/final_project/github/Data/images/validation/formatted_y_2\elementary3\P3_1_01_21180_50074.txt
저장 완료: C:/Users/boar2/Desktop/final_project/github/Data/images/validation/formatted_y_2\elementary3\P3_1_01_21186_50114.txt
저장 완료: C:/Users/boar2/Desktop/final_project/github/Data/images/validation/formatted_y_2\elementary3\P3_1_01_21187_50121.txt
저장 완료: C:/Users/boar2/Desktop/final_project/github/Data/images/validation/formatted_y_2\elementary3\P3_1_01_21187_50123.txt
저장 완료: C:/Users/boar2/Desktop/final_project/github/Data/images/validation/formatted_y_2\elementary3\P3_1_01_21193_50168.txt
저장 완료: C:/Users/boar2/Desktop/final_project/github/Data/images/validation/formatted_y_2\elementary3\P3_1_01_21236_50304.txt
저장 완료: C